# ML Modelling

In [1]:
#Importing Libraries
import sys
import os
import dvc.api
import numpy as np
import pandas as pd

In [2]:
#importing local scripts
sys.path.append(os.path.abspath(os.path.join('..')))
#Adding scripts path
#importing data_manipulator script
from scripts.data_manipulator import *

## Loading Data

In [55]:
#Importing the collected Data
path = 'data/AdSmartABdata.csv'
repo = 'https://github.com/DePacifier/abtest-mlops'
rev = 'chrome-mobile'
data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
collected_data = pd.read_csv(data_url)
collected_data.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [47]:
collected_data.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,response
0,5f8b891c-ae33-41a3-bd9f-7a8b5238b7ae,exposed,2020-07-07,3,Samsung SM-G973F,6,Samsung Internet,0
1,f79d0a9c-3d11-4145-aa4d-74bfb4dbc399,control,2020-07-06,3,Samsung SM-A105FN,6,Samsung Internet,1
2,6cf6656d-f47c-4224-a33f-3e71b16ae645,control,2020-07-04,7,Samsung SM-G970F,6,Samsung Internet,0
3,561ebe61-32e2-40fe-9913-c814c6cd49e1,exposed,2020-07-07,19,Samsung SM-G960F,6,Samsung Internet,1
4,2ed3be13-ed0e-4d36-88dd-12639db2c002,control,2020-07-03,15,Samsung SM-J415FN,6,Samsung Internet,0


## Split data by browser and platform_os, and version each split as a new version of the data in dvc.

In [25]:
### Spliting based on Browser
browser = collected_data.groupby('browser')
browser_size = browser.size()
browser_size

browser
Chrome                          2
Chrome Mobile                 695
Chrome Mobile WebView         227
Chrome Mobile iOS               1
Facebook                      156
Mobile Safari                  14
Mobile Safari UI/WKWebView      3
Samsung Internet              145
dtype: int64

In [26]:
# Selecting only browsers with more than 100 rows or users
def get_index_based_on_size(size:np.array,value:int) -> list:
    size = size.tolist()
    index_list = []
    for index in range(len(size)):
        if(size[index] >= value):
            index_list.append(index)

    return index_list

def get_df_of_each_group(grouped_df:pd.DataFrame, selected_index:list) -> list:
    size_series = grouped_df.size()
    df_list = []
    for index in selected_index:
        df = grouped_df.get_group(size_series.index[index])
        df_list.append(df.reset_index().drop('index', axis=1))

    return df_list


In [27]:
test_worthy_browser_list = get_index_based_on_size(browser_size.values, 100)
browser_dfs = get_df_of_each_group(browser, test_worthy_browser_list)
len(browser_dfs)

4

In [28]:
### Spliting based on Platform Os
platform = collected_data.groupby('platform_os')
platform_size = platform.size()
platform_size

platform_os
5      17
6    1226
dtype: int64

In [29]:
test_worthy_platform_list = get_index_based_on_size(platform_size.values, 100)
platform_dfs = get_df_of_each_group(platform, test_worthy_platform_list)
len(platform_dfs)

1

In [50]:
# Saving Each New Version and Adding it to DVC
# Browser Versions
browser_dfs[0].to_csv('../data/AdSmartABdata.csv', index=False)
# browser_dfs[1].to_csv('../data/AdSmartABdata.csv', index=False)
# browser_dfs[2].to_csv('../data/AdSmartABdata.csv', index=False)
# browser_dfs[3].to_csv('../data/AdSmartABdata.csv', index=False)
# browser_dfs[0].head(5)

# Add to dvc with a new tag "the name" => "chrome-mobile"
